# DRP Processing Notebook
Load the DRP slice, visualize the mean DRP, derive direction/mask maps, and locate peaks in a downsampled/blurred projection.

### Imports

In [ ]:
import numpy as np
import scipy
from matplotlib import pyplot as plt
import cv2

from src import ImagePack
from src.drp_direction import drp_direction_map, drp_mask_angle
from src.line_detection import hough_transform, find_hough_peaks


### Load images + DRP (2x2 angular slice)

In [ ]:
images = ImagePack(angle_slice=(2, 2), data_root="data", config_path="exp_param.yaml")


### Mean DRP plot

In [ ]:
ax = images.plot_drp(images.get_mean_drp(), cmap="jet")
plt.show()


### Direction map + mask

In [ ]:
mag_map, deg_map = drp_direction_map(images)
img_mask = drp_mask_angle(mag_map, deg_map, orientation=90, threshold=45)


### Normalize orientation to 0-255 and show

In [ ]:
img = (0.5 * (-np.sin(np.radians(deg_map)) + 1) * 255).astype(np.uint8)
plt.imshow(img, cmap="gray")
plt.show()


### Downsample first, then a single Gaussian blur instead of multiple box blurs

In [ ]:
img = img[::2, ::2]
img = cv2.GaussianBlur(img, (21, 21), sigmaX=8, sigmaY=8)
plt.imshow(img, cmap="gray")
plt.show()


### Column intensity profile

In [ ]:
img_stacked = img.mean(axis=0)
plt.plot(img_stacked)
plt.title("Stacked Image Intensity Profile")
plt.xlabel("Pixel Position")
plt.ylabel("Average Intensity")
plt.show()


### Peak detection (top 13 by height) and overlay

In [ ]:
peaks, _ = scipy.signal.find_peaks(img_stacked)
peaks_by_height = peaks[np.argsort(img_stacked[peaks])[::-1]]
keep = 13
plt.imshow(img, cmap="gray")
for peak in peaks_by_height[:keep]:
    plt.plot([peak, peak], [0, img.shape[0]], color="red", linewidth=1)
plt.show()
